#### Import our settings file that contains variables
#### Import tweepy, the library we use for connection to the twitter api
#### Import dataset, the library we use for managing our database
#### Import textblob, the library we use for simple text processing

In [ ]:
import settings
import tweepy
import dataset
from textblob import TextBlob

#### Handle Errors using SQLAlchemy
#### Import json to read and process the data returned from the Twitter API 

In [ ]:
from sqlalchemy.exc import ProgrammingError
import json

#### Connect to the SQLite database

In [ ]:
db = dataset.connect(settings.CONNECT)

#### We create a subclass of the StreamListener class and implement our custom logic. 
#### We override the on_status method which will get data when users post new tweets.
#### We also override the on_error method to handle errors coming from the Twitter API.

In [ ]:
class StreamListener(tweepy.StreamListener):
    def on_status(self, status):
        # If the retweeted_status property exists, then ignore the tweet.
        if status.retweeted:
            return
        # Description for the user who tweeted (status.user.description).
        description = status.user.description
        # Location for the user who tweeted (status.user.location).
        loc = status.user.location
        # Text of the tweet (status.text).
        text = status.text
        # Geographic coordinates for the tweet (status.coordinates).
        coords = status.coordinates
        # The screen name of the user (status.user.screen_name).
        name = status.user.screen_name
        # Creation date-time for this user's account (status.user.created_at).
        user_created = status.user.created_at
        # Follower count for this user (status.user.followers_count).
        followers = status.user.followers_count
        # The unique id for this tweet in Twitter's database (status.id_str).
        id_str = status.id_str
        # Creation date-time for this tweet (status.created_at).
        created = status.created_at
        # Retweet count for this tweet (status.retweet_count).
        retweets = status.retweet_count
        # Background color for this user's profile (status.user.profile_background_color).
        bg_color = status.user.profile_background_color
        # use TextBlob to process the text of the tweet
        blob = TextBlob(text)
        # use TextBlob to get a sentiment score on the tweet
        sent = blob.sentiment
        # converted to json object
        if coords is not None:
            coords = json.dumps(coords)
        table = db[settings.TABLE_NAME]
        # insert all the data we obtained into the table
        try:
            table.insert(dict(
                user_description=description,
                user_location=loc,
                coordinates=coords,
                text=text,
                user_name=name,
                user_created=user_created,
                user_followers=followers,
                id_str=id_str,
                created=created,
                retweet_count=retweets,
                user_bg_color=bg_color,
                polarity=sent.polarity, # negativity or positivity of the tweet, on a -1 to 1 scale.
                subjectivity=sent.subjectivity, # 0 = tweet is very objective, and 1 = tweet is very subjective.
            ))
        except ProgrammingError as err:
            print(err)
    
    def on_error(self, status_code):
        # status code 420 indicates we are being rate limited by the API
        if status_code == 420:
            return False

#### Using the OAuthHandler we establish our credentials with Twitter and create an 'auth' object
#### Using 'auth' we create an API object to pull data from Twitter

In [ ]:
auth = tweepy.OAuthHandler(settings.TWITTER_CONSUMER_KEY, settings.TWITTER_CONSUMER_SECRET)
auth.set_access_token(settings.TWITTER_ACCESS_TOKEN, settings.TWITTER_ACCESS_SECRET)
api = tweepy.API(auth)

#### First we create an instance of our StreamListener class. Then we create an instance of the tweepy Stream class.
#### Here, pass in authentication credentials (api.auth) to connect and pass stream_listener as listener.
#### The filter method starts streaming tweets and pass in a list of terms (TRACK_TERMS) to filter on.

In [ ]:
stream_listener = StreamListener()
stream = tweepy.Stream(auth=api.auth, listener=stream_listener)
stream.filter(track=settings.TRACK_TERMS)